## Goals
To scrape the DepartureVision board every minute and store the data into a database for further analyses. 

**This is a WIP



In [2]:
#imports for web scraping
import urllib
from bs4 import BeautifulSoup
import requests 

#Data analysis imports
import pandas as pd
import numpy as np

#Vizualization Import
import matplotlib.pyplot as plt

#Data cleaning imports
import re
import time

In [3]:
#Scrape the data

url = 'http://dv.njtransit.com/mobile/tid-mobile.aspx?sid=NY'  #this is the DepartureVision (DV) url for train information originating at Penn Station


def parseDV(url):   #create a function that takes the DV url and outputs the data in a list
  get = urllib.request.urlopen(url)
  html = get.read()

  soup = BeautifulSoup(html, 'html.parser')

  dimensions = soup.find_all('td')

  rows = soup.find_all('tr') #rows is a list with html code, list includes 40 items - aka 40 rows with the data

  trains = []
  exceptions = []
  for x in rows[2:]:
    split_rows = x.get_text().split('</p></td>') #get_text() extracts one long string with \n's, etc, and split() breaks a string with the specified delimiter and returns a LIST of strings
    for i in split_rows:
      splitted = i.strip().split('\n')      #.strip() removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
      splitted[0] = splitted[0][:5] ##remove the \r
      if len(splitted)>7:
        exceptions.append(splitted)
      elif len(splitted)<=7:
        splitted[1] = 'unavailable'
      if splitted[3]=='':
        splitted[3] = 'track unavailable'
      if len(splitted)==6:
        splitted.append('status unavailable')
      current_time = time.strftime("%m-%d-%Y %I:%M:%S %p", time.localtime())
      splitted.append(current_time)
    trains.append(splitted)
  return trains


In [4]:
#Create a Dataframe from the scraped data

def createDF(trains):   #turns the outputted list into a dataframe
  df = pd.DataFrame(trains)
  df.columns = ['DEPARTS', 'Extra', 'TO','TRACK','LINE','TRAIN','STATUS','TIME']
  del(df['Extra'])
  return df

In [5]:
dataframe1 = createDF(parseDV(url))
df_dropped = dataframe1.drop_duplicates('TRAIN').reset_index(drop=True)
df_dropped

,DEPARTS,TO,TRACK,LINE,TRAIN,STATUS,TIME
0,11:15,Philadelphia ✈,5,REGIONAL,A639,BOARDING,08-21-2018 11:12:32 PM
1,11:18,Long Branch-BH -SEC ✈,10,No Jersey Coast,3297,BOARDING,08-21-2018 11:12:32 PM
2,11:41,MSU -SEC,track unavailable,Montclair-Boonton,6299,status unavailable,08-21-2018 11:12:32 PM
3,12:02,Dover -SEC,track unavailable,Morristown Line,6683,status unavailable,08-21-2018 11:12:32 PM
4,12:05,Long Branch -SEC ✈,track unavailable,No Jersey Coast,3201,status unavailable,08-21-2018 11:12:32 PM
5,12:08,Raritan -SEC,track unavailable,Raritan Valley,5101,status unavailable,08-21-2018 11:12:32 PM
6,12:14,Trenton -SEC ✈,track unavailable,Northeast Corrdr,3897,status unavailable,08-21-2018 11:12:32 PM
7,12:34,MSU -SEC,track unavailable,Montclair-Boonton,6201,status unavailable,08-21-2018 11:12:32 PM
8,12:56,Dover -SEC,track unavailable,Morristown Line,6601,status unavailable,08-21-2018 11:12:32 PM
9,1:00,Long Branch -SEC ✈,track unavailable,No Jersey Coast,3205,status unavailable,08-21-2018 11:12:32 PM
